# 파이널 프로젝트 Overview
* 마지막 문제풀이입니다! 데이터리안 블로그 GA 데이터를 분석하면서 이번 강의에서 배운 판다스의 다양한 기능들을 사용해봅시다.
* 문제를 본격적으로 풀기 전에 아래 코드를 실행해 pandas 라이브러리를 임포트하고 데이터를 읽어와주세요.

In [1]:
import pandas as pd

In [3]:
ga = pd.read_csv('https://raw.githubusercontent.com/datarian-kr/pandas/main/data/ga_encrypted.csv')

# 데이터 설명

이 데이터는 [데이터리안의 테크 블로그](https://datarian-kr.github.io/) GA(Google Anlaytics) 데이터의 일부입니다. 데이터리안은 GA를 이용해 테크 블로그 안에서 사용자들이 어떤 페이지를 보는지, 어떤 요소를 클릭하는지 등 사용자 로그를 분석합니다.

데이터리안 블로그의 'Blog' 탭에 들어가면 멤버들이 쓴 글을 보실 수 있습니다. 이 글을 소비하는 사용자들의 패턴을 분석하여 앞으로 데이터리안이 어떤 글을 더 써야할지, 어떤 글을 사용자들이 흥미롭게 생각했는지 등을 파악해보려고 합니다.

<div>
<img src="https://github.com/datarian-kr/pandas/blob/main/img/blog_main.png?raw=true" width="500"/>
</div>

**컬럼 상세**
* event_date_kst: 이벤트가 일어난 날짜 (KST)
* user_pseudo_id: 암호화 된 사용자 아이디
* event_timestamp_kst: 이벤트가 일어난 시각 (KST)
* user_first_touch_timestamp_kst: 사용자가 처음 사이트를 방문한 시각 (KST)
* event_name: 이벤트 이름
* key: 이벤트 매개변수의 키
* string_value: 이벤트 매개변수의 문자열 값
* int_value: 이벤트 매개변수의 정수 값
* geo_continent: 이벤트가 발생한 대륙
* geo_country: 이벤트가 발생한 국가
* traffic_source_medium: 사용자를 최초로 획득한 매체
* traffic_source: 사용자를 최초로 획득한 네트워크의 이름

**참고**  
위 데이터는 BigQuery로 내보낸 GA4의 속성 데이터입니다. 데이터의 형식 및 스키마에 대한 더 자세한 설명은 `BigQuery Export 스키마` 문서를 참고하세요. 각 이벤트가 수집되는 기준은 `[GA4] 자동으로 수집되는 이벤트` 문서에서 찾아보실 수 있습니다.
* [BigQuery Export 스키마](https://support.google.com/analytics/answer/7029846?hl=ko)
* [[GA4] 자동으로 수집되는 이벤트](https://support.google.com/analytics/answer/9234069)

# Q1. 탐색적 데이터 분석

데이터를 탐색해봅시다.

**아래의 간단한 질문들에 답하며 EDA를 시작해도 좋습니다.**
* 총 몇 개의 데이터가 들어있나요?
* 몇 일치의 데이터가 들어있나요?
* `event_name`, `key`, `geo_continent` 등 컬럼에는 각각 어떤 데이터들이 들어있나요?
* 일자 별로 몇 명의 유저가 방문했나요?
* 블로그 글을 조회하거나, 읽은 것과 관련한 로그는 어떤 조건으로 찾을 수 있나요? (중요!)

In [5]:
# Answer
ga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12457 entries, 0 to 12456
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   event_date_kst                  12457 non-null  object 
 1   user_pseudo_id                  12457 non-null  object 
 2   event_timestamp_kst             12457 non-null  object 
 3   user_first_touch_timestamp_kst  12457 non-null  object 
 4   event_name                      12457 non-null  object 
 5   key                             12457 non-null  object 
 6   string_value                    7050 non-null   object 
 7   int_value                       5407 non-null   float64
 8   geo_continent                   12457 non-null  object 
 9   geo_country                     12457 non-null  object 
 10  traffic_source_medium           12457 non-null  object 
 11  traffic_source                  12457 non-null  object 
dtypes: float64(1), object(11)
memory

# Q2. 블로그 글과 관련한 로그 필터링

전체 데이터 중 분석에 필요한 로그만 필터링합니다. 아래 조건을 충족하는 데이터를 선택해서 `pages` 데이터 프레임에 저장해주세요.

**조건**
1. event_name이 `page_view`, `scroll`, `user_engagement` 중에 하나일 것
2. 해당 이벤트의 파라미터(key 컬럼) 중 `page_title` 만 선택

**이벤트 상세**

상단에 `[GA4] 자동으로 수집되는 이벤트` 문서 참고

* first_visit: 사용자가 웹사이트에 처음 방문
* session_start: 사용자가 앱 또는 웹사이트에 참여할 때
* page_view: 페이지가 로드될 때
* scroll: 사용자가 각 페이지에서 처음으로 하단에 도달할 때(세로 기준으로 페이지의 90% 이상이 표시될 때)
* user_engagement: 1초 이상 웹페이지에 머물렀을 때
* click: 사용자가 현재 도메인에서 나가는 링크를 클릭할 때

In [32]:
# Answer
pages = ga[((ga['event_name'] == 'page_view')|(ga['event_name'] == 'scroll')|(ga['event_name'] == 'user_engagement'))]
pages = pages[pages['key']=='page_title']

pages

,event_date_kst,user_pseudo_id,event_timestamp_kst,user_first_touch_timestamp_kst,event_name,key,string_value,int_value,geo_continent,geo_country,traffic_source_medium,traffic_source
16,2021-08-05,2e7e591e5b2b75de74baf0ae1ae6671d83240388,2021-08-05 23:55:07,2021-08-05 23:55:07,page_view,page_title,내가 만약 카톡 선물하기의 분석가라면,NaN,Asia,South Korea,(none),(direct)
19,2021-08-05,2e7e591e5b2b75de74baf0ae1ae6671d83240388,2021-08-05 23:55:25,2021-08-05 23:55:07,scroll,page_title,내가 만약 카톡 선물하기의 분석가라면,NaN,Asia,South Korea,(none),(direct)
30,2021-08-05,2e7e591e5b2b75de74baf0ae1ae6671d83240388,2021-08-05 23:57:48,2021-08-05 23:55:07,user_engagement,page_title,내가 만약 카톡 선물하기의 분석가라면,NaN,Asia,South Korea,(none),(direct)
45,2021-08-05,93b652814b54bc68233f5f3ab7f021a778b575d6,2021-08-05 22:07:52,2021-08-05 22:07:52,page_view,page_title,Datarian,NaN,Asia,South Korea,(none),(direct)
47,2021-08-05,93b652814b54bc68233f5f3ab7f021a778b575d6,2021-08-05 22:07:59,2021-08-05 22:07:52,scroll,page_title,Datarian,NaN,Asia,South Korea,(none),(direct)
...,...,...,...,...,...,...,...,...,...,...,...,...
12336,2021-08-10,6815e666f5e45dbc148219cc7e89b8f61fc9f244,2021-08-10 10:31:08,2021-08-10 10:11:22,user_engagement,page_title,Datarian,NaN,Asia,South Korea,(none),(direct)
12339,2021-08-10,6815e666f5e45dbc148219cc7e89b8f61fc9f244,2021-08-10 10:55:26,2021-08-10 10:11:22,page_view,page_title,Datarian,NaN,Asia,South Korea,(none),(direct)
12421,2021-08-10,6815e666f5e45dbc148219cc7e89b8f61fc9f244,2021-08-10 10:56:48,2021-08-10 10:11:22,user_engagement,page_title,Datarian,NaN,Asia,South Korea,(none),(direct)
12445,2021-08-10,ef9eb5753c4c556f95148152f327e5fa37929efc,2021-08-10 20:13:29,2021-08-10 20:13:29,page_view,page_title,데이터 분석을 위한 SQL 참고도서 추천,NaN,Asia,South Korea,referral,channeltab-godfather.onkakao.net


# Q3. 사람들이 가장 많이 본 페이지는 무엇일까요?

사람들이 가장 많이 본 페이지가 뭔지 알아보고 데이터리안이 앞으로 블로그에 어떤 글을 더 써야할지 생각해봅시다. 아래 스텝에 따라 진행해주세요.

* Step 1. `pages` 데이터 프레임에서 '페이지 조회' 이벤트만 필터링하여 `page_view` 데이터 프레임을 만들어주세요.
* Step 2. `page_view` 을 가지고 데이터 기간 동안 어떤 페이지가 몇 번 조회되었는지, 몇 명이 조회하였는지 계산하고 이를 `page_view_cnt` 데이터 프레임에 저장해주세요.
  - Step 2 Output 예시

| string_value                                |   count |   nunique |
|:--------------------------------------------|--------:|----------:|
| 2020 현업 분석가 3인 선데이 아점 토크 _ 1화 |       3 |         3 |
| 2020 현업 분석가 3인 선데이 아점 토크 _ 2화 |       1 |         1 |
| Blog                                        |      94 |        47 |

* Step 3. 위 데이터를 페이지가 많이 조회 된 순서대로 정렬해주세요.

In [41]:
# Answer
pages[pages['event_name']=='page_view'].groupby('string_value')['event_timestamp_kst','user_pseudo_id'].nunique()

/var/folders/83/jp6r_0zj1p39llmq2gz9rq380000gn/T/ipykernel_17226/161362929.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  pages[pages['event_name']=='page_view'].groupby('string_value')['event_timestamp_kst','user_pseudo_id'].nunique()


,event_timestamp_kst,user_pseudo_id
string_value,,
2020 현업 분석가 3인 선데이 아점 토크 _ 1화,3,3
2020 현업 분석가 3인 선데이 아점 토크 _ 2화,1,1
Blog,94,47
Datarian,117,63
Grid Search vs Random Search,1,1
Members,87,32
[백문이불여일타] SQL 시리즈 강의 가이드,5,4
내가 만약 카톡 선물하기의 분석가라면,34,26
데이터 분석가 빙고,1,1


### 질문

* 질문1: 가장 많이 조회된 페이지는 어디이고, 데이터 기간 동안 몇 번 조회되었나요?
* 답변1: 

<br>

* 질문2: 데이터리안 블로그의 구조에서 '탭'에 속하는 'Datarian', 'Blog', 'Member'를 제외한 페이지들은 블로그 글입니다. 어떤 글이 가장 많이 읽혔나요? 데이터 기간 동안 몇 번 조회되었나요?
* 답변2: 

# Q4. 사람들이 가장 많이 읽은 페이지는 무엇일까요?

위에서 살펴본 pageview 로그는 단순히 그 '페이지를 조회했다'를 의미합니다. 그러면 사람들이 글을 실제로 읽었는지 확인해보려면 어떤 방법들이 있을 수 있을까요?

그 페이지에 얼마나 머물렀는지, 스크롤을 얼마나 내렸는지, 글 안의 링크들을 클릭해 보았는지 등으로 사람들이 그 컨텐츠를 실제로 소비했는지를 가늠해 볼 수 있을겁니다. 이번 문제에서는 'page_view'와 함께 'user_engagement', 'scroll' 로그를 함께 분석해봅시다.

**[참고] 이벤트 정의**

상단에 `[GA4] 자동으로 수집되는 이벤트` 문서에서 더 자세한 내용을 보실 수 있습니다.
* page_view: 페이지가 로드될 때
* scroll: 사용자가 각 페이지에서 처음으로 하단에 도달할 때(세로 기준으로 페이지의 90% 이상이 표시될 때)
* user_engagement: 1초 이상 웹페이지에 머물렀을 때

**아래 스텝에 따라서 진행해주세요.**

* Step 1. `pages` 데이터 프레임을 활용하여 페이지 별 조회수, 스크롤 90% 이상 히트 횟수, 페이지에서 1초 이상 머무른 횟수를 집계해주세요. 집계 결과는 `pages_pivot` 변수에 저장하세요.

  - Step 1 Output 예시

| string_value                                |   page_view |   scroll |   user_engagement |
|:--------------------------------------------|------------:|---------:|------------------:|
| 2020 현업 분석가 3인 선데이 아점 토크 _ 1화 |           3 |        1 |                 3 |
| 2020 현업 분석가 3인 선데이 아점 토크 _ 2화 |           1 |        0 |                 1 |
| Blog                                        |          94 |       22 |                82 |

<br>

* Step 2. 데이터 해석의 편의를 위해 `pages_pivot` 데이터 프레임을 조회수가 많은 기준으로 정렬하여 `pages_pivot` 데이터 프레임에 다시 저장합니다.

<br>

* Step 3. 블로그 글에 노출된 사람들 중 그 글을 실제로 읽은 사람이 얼마나 되는지, 다 읽은 사람이 얼마나 되는지 비율을 계산하고 싶습니다. `page_view`, `scroll`, `user_engagement` 컬럼을 이용하여 비율을 계산하고 `scroll %`, `user engagement %` 컬럼을 추가하세요.

  - Step 3 Output 예시

| string_value   |   page_view |   scroll |   user_engagement |   scroll % |   user engagement % |
|:---------------|------------:|---------:|------------------:|-----------:|--------------------:|
| Datarian       |         117 |       68 |                97 |    58.1197 |             82.906  |
| Blog           |          94 |       22 |                82 |    23.4043 |             87.234  |
| Members        |          88 |       26 |                64 |    29.5455 |             72.7273 |

In [65]:
# Answer
pivot_table = pd.pivot_table(pages, index='string_value', columns='event_name',values='event_timestamp_kst',aggfunc='count')
pivot_table = pivot_table.sort_values(by='page_view', ascending=False)
pivot_table['scroll'] = pivot_table['scroll']/pivot_table['page_view']
pivot_table['user engagement'] = pivot_table['user_engagement']/pivot_table['page_view']
pivot_table.head()

event_name,page_view,scroll,user_engagement,user engagement
string_value,,,,
Datarian,117.0,0.581197,97.0,0.829060
Blog,94.0,0.234043,82.0,0.872340
Members,88.0,0.295455,64.0,0.727273
데이터 분석을 위한 SQL 참고도서 추천,86.0,0.581395,58.0,0.674419
내가 만약 카톡 선물하기의 분석가라면,34.0,0.529412,21.0,0.617647


### 질문

* 질문1: 페이지의 하단 90% 까지 내렸을 때 `scroll` 로그가 기록됩니다. 이 로그를 보고 글의 완독률을 대략 파악할 수 있겠지요! '데이터 분석을 위한 SQL 참고도서 추천' 글의 완독률은 어느정도 인가요? 다른 글과 비교해 높은 편인가요 낮은 편인가요?
* 답변1:

<br>

* 질문2: 페이지에서 1초 이상 머물 경우 `user_engagement` 로그가 기록됩니다. `page_view` 로그는 있으나 `user_engagement` 로그가 없는 경우를 어떻게 해석할 수 있을까요? '데이터 분석을 위한 SQL 참고도서 추천'의 `user engagement` 비율은 얼마이며 이를 어떻게 개선할 수 있을까요?
* 답변2:

# Q5. 여러분들의 분석 아이디어를 들려주세요.

여러분들의 분석 내용을 인프런 블로그 또는 개인 블로그에 `#데이터리안` `#판다스공부` `#TIL` 해시태그로 공유해주세요! `velog.io`, `brunch`, `medium` 등 개인 블로그에 올리신 분들은 인프런 블로그로 링크를 올려주시면 저희가 검색해서 찾아갈 수 있을 것 같아요.

분석 내용 뿐 아니라 노트북 자체를 공유하고 싶은 분들은 아래 예시를 참고해주세요.
* [nbviewer 예시](https://nbviewer.jupyter.org/github/dataitgirls3/Teaching-Materials/blob/master/2019-09-09%20Pandas%20%EC%88%98%EC%97%85%20%EB%B3%B4%EC%B6%A9%EC%9E%90%EB%A3%8C.ipynb)
* [github 예시](https://github.com/dataitgirls3/Teaching-Materials/blob/master/Comprehension.ipynb)

인프런 블로그에 올리실 때 `#데이터리안` `#판다스공부` 해시태그 다는거 잊지 말아주세요!

In [ ]:
# 추가 분석

# 수고하셨습니다!